In [ ]:
import pandas as pd
import yfinance as yf

#### Explore Potential Industry

In [ ]:
# !wget -P ../data/external/ https://pages.stern.nyu.edu/~adamodar/pc/datasets/betas.xls
# !wget -P ../data/external/ https://pages.stern.nyu.edu/~adamodar/pc/datasets/indname.xls

In [ ]:
df = pd.read_excel(
    "../data/external/betas.xls",
    sheet_name="Industry Averages",
    skiprows=9,
    skipfooter=2,
)

df.columns = df.columns.str.lower().str.strip().str.replace(" ", "_")

df.sort_values(by="beta", ascending=False, ignore_index=True, inplace=True)

industry = df["industry_name"][0]
country = "United States"

In [ ]:
df = pd.read_excel(
    "../data/external/indname.xls",
    sheet_name="By Industry",
)

df.columns = df.columns.str.lower().str.strip().str.replace(" ", "_") \
    .str.replace(r"[^a-zA-Z0-9_]", "_", regex=True)

tickers = df.query("industry_group == @industry and country == @country") \
    ["exchange_ticker"].apply(lambda x: x.split(":")[-1]).values

#### Download Market Data

In [ ]:
cagrs = []

for ticker in tickers:
    equity = yf.Ticker(ticker)
    df = equity.history(start="2010-01-01")
    df.to_parquet(f"../data/raw/equities/{ticker}.parquet")

    v = df[df.index.year.isin(range(2010, 2024+1))]["Close"]
    t = max(df.index.year[-1] - df.index.year[0], 1)

    cagrs.append({
        "ticker": ticker,
        "cagr": (v.iat[-1] / v.iat[0]) ** (1 / t) - 1
    })

max(cagrs, key=lambda x: x["cagr"])

In [ ]:
ticker = "^IRX"
bill = yf.Ticker(ticker)
df = bill.history(start="2010-01-01")
df.to_parquet(f"../data/raw/bills/{ticker}.parquet")

In [ ]:
ticker = "XHB"
index = yf.Ticker(ticker)
df = index.history(start="2010-01-01")
df.to_parquet(f"../data/raw/indices/{ticker}.parquet")

#### Investigate Inconsistent Data

In [ ]:
df_0529 = pd.read_parquet("../data/yfinance/HD_20250529.parquet") \
    .assign(version="0529")
df_0602 = pd.read_parquet("../data/yfinance/HD_20250602.parquet") \
    .assign(version="0602")
df_0610 = pd.read_parquet("../data/yfinance/HD_20250610.parquet") \
    .assign(version="0610")

df = yf.Ticker("HD").history(start="2025-01-01")
df.query("Dividends != 0").tail()

In [ ]:
df_dup = pd.concat([df_0529, df_0602, df_0610]) \
    .drop_duplicates(subset=["Open", "High", "Low", "Close"], keep=False)

df_dup["date"] = df_dup.index.strftime("%Y-%m-%d")
df_dup = df_dup.merge(
    df_dup.groupby("date", as_index=False).agg(cnt=("version", "count"))
)

dates = df_dup.query("cnt == 3")["date"].values

df_dup.query("date == @dates[-1]")

In [ ]:
df_dup = pd.concat([df_0529, df_0602, df_0610]) \
    .drop_duplicates(subset=["Open", "High", "Low", "Close", "Volume"], keep=False)

df_dup["date"] = df_dup.index.strftime("%Y-%m-%d")
df_dup = df_dup.merge(
    df_dup.groupby("date", as_index=False).agg(cnt=("version", "count"))
)

dates = df_dup.query("cnt == 3")["date"].values

df_dup.query("date == @dates[-1]")